# General information

Import the previously generated scenarios

In [ ]:
# Name of the data file 
datafile = "usecase_initialdata.xlsx"

In [ ]:
import tools
import analyse_bdd_general
import analyse_spmf
import TraitementDataconflit
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


a = TraitementDataconflit.tablecreation(datafile)
data_decision = a[3]
data_opinions = a[0]
data_positions = a[1]
data_user = a[2]
data_initialstate = a[6]
data_logicalconstraint = a[4]
data_pplconstraint = a[5]
data_goals = a[7]

D = tools.dataload('stockage_d.pkl')
state_dict = tools.dataload('stockagedict_etat.pkl')
scenarios = tools.dataload('stockage_scenarios.pkl')
system = tools.dataload('stockage_systeme.pkl')

scenariosconflict = tools.scenarioobjatteint(scenarios, "conflit logique")
scenariosgoals = tools.scenarioobjatteint(scenarios, "objectif(s) atteint(s)")

In [ ]:
print(system.pp)

print the main statistics resulting from the data

In [ ]:
print(analyse_bdd_general.statistiques(scenarios, state_dict, data_opinions))

# Scenarios grouping

## Grouping accoding to value(s) of variable(s)

As an example we look here for all the scenarios including a sanitary crisis

In [ ]:
#look for scenarios with a specific feature (value)
    #dict of variables and their values
value = {'CriseSanitaire':['Oui_cs']}

In [ ]:
a = tools.value_in_scenarios(value, scenarios, state_dict)
tools.allscenariostoreadbetter('result3_copy2.txt', a, state_dict)
len(a)

## Grouping by scenarios ending criteria

In [ ]:
import matplotlib.pyplot as plt
import copy

a = analyse_bdd_general.var_vs_var_conflict(scenariosconflict)

var = list(system.v.keys())

fig = plt.figure()
min = 0
max = len(scenariosconflict)
dmin =0
dmax=1000

var2 = copy.deepcopy(var)
points = []
ocs = []
for v1 in var :
    for v2 in var :
        tempooc = a.get(v1,0)
        if tempooc != 0 :
            oc = tempooc.get(v2,0)
            t=(oc-min)/(max-min)
            taille = t*(dmax)+(1-t)*dmin
            if (not [v2, v1, taille] in points) and (not [v1, v2, taille] in points):
                points.append([v2, v1, taille])
                ocs.append(oc)
        else:
            points.append([v2, v1, 0])
print(ocs)

v2s, v1s, tailles = [], [], []
for point in points:
    v2s.append(point[0])
    v1s.append(point[1])
    tailles.append(point[2])
print(tailles)
print(v2s)
fig, ax = plt.subplots()

plt.scatter(v2s,v1s,s=tailles,marker='o',color='olivedrab')
for i in range(len(tailles)):
    if tailles[i]!=0 :
        plt.annotate(ocs[i],xy=(v2s[i],v1s[i]), textcoords='offset points', xytext=(0,13))

plt.ylabel('Variables')
plt.xlabel('Variables')
plt.xticks(rotation=80)

ax.set_ylim(-0.5, len(var)-0.5)
ax.set_xlim(-0.5, len(var)-0.5)
ax.set_yticks(var)
ax.set_xticks(var)


image_format = 'svg' # e.g .png, .svg, etc.
image_name = 'ratio_varvar_conflit.svg'

fig.savefig(image_name, format=image_format, dpi=1200)

## Grouping by repected principles

In [ ]:
#respected principle
resp_principle = "SatisfactionClient"

In [ ]:
#We take into account only the decisions taken by the user

a = analyse_bdd_general.respected_pp(scenarios, resp_principle,0,system, state_dict, data_positions, data_opinions)
tools.allscenariostoreadbetter('result3_copy2.txt', a, state_dict)
print(len(a))
print(a[0])

In [ ]:
#We take into account every decisions of each agents

b = analyse_bdd_general.respected_pp_allact(scenarios, resp_principle,0,system, state_dict, data_positions, data_opinions)
tools.allscenariostoreadbetter('result3_copy2.txt', b, state_dict)
print(len(b))
print(b[0])

In [ ]:
import numpy as np
data_parallel = tools.data_parallel_scenario(b, state_dict, system)
# an example as also been made using best[0].get("WealthCreation") instead of b in the following code


dim_list =[]

id_list=list(range(1,len(b)+1)) 
ids = np.tile(id_list,6)

dim_list.append({'label':'ids', 'values': ids})

dim_list.append({'label':'var','values':data_parallel[1]})


for i in data_parallel[2] :
    dim_list.append({'label':i,'values': data_parallel[2].get(i)})

print(data_parallel[1])


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px


#listescenario
liste40=[]
for i in range(1,40): #40 is the number of scenarios in the example using b
    liste40.append(i)

#listevar
list_var = list(system.v.keys())
list_nbvar = [*range(1,len(list_var)+1,1)]
dict_var = {}
var = data_parallel[1]
index = 1
#dict var nb de reference
for v in list_var :
    dict_var.update({v:index})
    index +=1
newlistvar = []
#turn listvar into a list of numbers
for v2 in var :
    newlistvar.append(dict_var.get(v2))

#listeval
liste_val = []
for var in system.v : 
    for val in system.v.get(var).valeurs:
        liste_val.append(val)
liste_nbval = [*range(1,len(liste_val)+1,1)]
dict_val = {}
index = 1
for val in liste_val :
    dict_val.update({val:index})
    index +=1

dim_list=[]
dim_list.append(dict(range = [1,40], 
                 label = 'Scenarios ID', values = ids))
dim_list.append(dict(range = [1,len(list_var)],
                 tickvals = list_nbvar,
                 label = 'Variables', values = newlistvar,
                 ticktext = list_var))

for i in data_parallel[2] :
    newlistval = []
    for val2 in data_parallel[2].get(i) :
        if val2 == None :
            newlistval.append(0)
        else :
            newlistval.append(dict_val.get(val2))
    dim_list.append({'label':i, 'range':[0,len(liste_val)],
                     'tickvals' : liste_nbval, 'values': newlistval,
                     'ticktext':liste_val})

color = id_list;
fig = go.Figure(data=
    go.Parcoords(
        line={'color':ids, 'colorscale':'Portland'}, 
        dimensions = list(dim_list),
        unselected = dict(line = dict(color = 'green', opacity = 0.05))
    )
)

fig.write_image("images/test.svg")
fig.write_image("images/test.pdf")
fig.write_image("images/test.png")
fig.show()

# Conflicts management and avoidance

## Conflicts statistics

In [ ]:
statconflit = analyse_bdd_general.statconflit(scenariosconflict)
print("\nStatistics on the variables responsible for conflicts")
print(statconflit[0])
print("\nStatistics on the decisions responsible for conflicts")
print(statconflit[1])

Statistics on variables

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
variable = list(statconflit[0].keys())
ratio = list(statconflit[0].values())
width = 0.25

ax.bar(variable,ratio, width, color='olivedrab')
ax.bar_label(ax.containers[0], label_type='edge')

plt.ylabel('Occurence ratio in conflicts')
plt.xlabel('Variables')


image_format = 'png' # e.g .png, .svg, etc.
image_name = 'ratio_variable_conflit.png'

plt.savefig(image_name, format=image_format, dpi=1200)

plt.show()

Statistics on decisions

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

variable = list(statconflit[1].keys())
ratio = list(statconflit[1].values())
width = 0.35

ax.bar(variable,ratio, width, color='powderblue')
ax.bar_label(ax.containers[0], label_type='edge')

plt.ylabel('Occurence ratio in conflict')
plt.xlabel('Decisions')
plt.xticks(rotation=80)
image_format = 'png' # e.g .png, .svg, etc.
image_name = 'ratio_decision_conflit.png'

fig.savefig(image_name, format=image_format, dpi=1200)

plt.show()

In [ ]:
print(analyse_bdd_general.Newgraphettppe(scenarios,system, state_dict, data_positions, data_opinions))


In [ ]:
print(analyse_bdd_general.Linkinggraphettppe(scenarios, system, state_dict, data_positions, data_opinions))

## Data mining algorithms on conflicts analysis

In [ ]:
#Prepare the files that are going to be analyse by the next data mining algorithms 
#(RuleGrowth and PrefixSpan)
analyse_spmf.inputBDDspmf(scenarios, D, state_dict)
analyse_spmf.inputspmfprincipe(scenarios, system)
print("Database generated")

### Sequential rule mining

<blockquote>Use spmf to produce the file for the following analysis part (<a href="https://www.philippe-fournier-viger.com/spmf/index.php?link=download.php">https://www.philippe-fournier-viger.com/spmf/index.php?link=download.php</a>) <br>
        <ul><li>the example files have been produced using the following parameter for the RuleGrowth algorithm : (min sup = 0.005 and minconf = 0.6) and the inputBDD file generated just above</li>
        <li>the result for the example is stored in the file : output_miniusecase1_usereasyflight.txt </li></ul></blockquote>

In [ ]:
#Generate the file using the algorithm RuleGrowth with the file inputBDD
dataoutputspmf = analyse_spmf.outputspmf('output_miniusecase1_usereasyflight.txt')
listconflitspmf = analyse_spmf.listconflitspmf(dataoutputspmf)
listobjectifspmf = analyse_spmf.objectifspmf(dataoutputspmf)

In [ ]:
print("The set of decisions giving conflicts with the highest confidence/sufficiency: ")
maxconf = analyse_spmf.max_seq(listconflitspmf, 2, 'No')
print(maxconf[0])

In [ ]:
if maxconf[1] == 1 :
        print("There is one or more set of decisions that are sufficient for a conflict to happen", max[0])
else:    
    print("There is no set of decisions that is sufficient for a conflict to happen.")

In [ ]:
print ("The smallest set of decision with the maximum probability for a conflict to happen is: ")
sequencemin = analyse_spmf.seqmin(maxconf[0])
print(sequencemin)

In [ ]:
#to know if the smallest sequence with the highest sufficiency is necessary
necessary = analyse_spmf.set_necessary(sequencemin, len(scenariosconflict))
print(necessary)

In [ ]:
#to look for the most necessary sequences
necessary2 = analyse_spmf.set_necessary(listconflitspmf, len(scenariosconflict))
print(necessary2)

### Sequential pattern mining

<blockquote>Use spmf to produce the file for the following analysis part (<a href="https://www.philippe-fournier-viger.com/spmf/index.php?link=download.php">https://www.philippe-fournier-viger.com/spmf/index.php?link=download.php</a>)<br>
        <ul><li>the example files have been produced using the following parameter for the PrefixSpan algorithm : minsup = 0.1 and the inputBDD file </li>
        <li>the result for the example is stored in the file : output_miniusecase_usereasyflight_prefixspan.txt </li></ul></blockquote>

In [ ]:
#Generate the file using PrefixSpan with the file inputBDD
dataoutputspmf2 = analyse_spmf.outputspmf2('output_miniusecase_usereasyflight_prefixspan.txt')
listconflitspmf2 = analyse_spmf.listconflitspmf2(dataoutputspmf2, "conflitlogique") #end : "objectif(s)atteint(s)", "conflitlogique"
maxsup = analyse_spmf.max_seq(listconflitspmf2, 1, 'Yes') #put 'yes' if you want as results, sections with a length higher than 1
necessary_PS = analyse_spmf.set_necessary(maxsup[0], len(scenariosconflict))
print(necessary_PS)

In [ ]:
listedecisionscenarios = tools.listedecisionscenario1(scenarios)
sufficiency_PS = analyse_spmf.set_sufficiency(necessary_PS[1], listedecisionscenarios)
print(sufficiency_PS)

In [ ]:
sufficiency_PS2 = analyse_spmf.set_sufficiency(listconflitspmf2, listedecisionscenarios)
print(sufficiency_PS2)

le plus petit parmi les motifs à plus haute confiance

In [ ]:
print(analyse_spmf.seqmin(sufficiency_PS2[1]))

le plus haut support avec la plus haute confiance

In [ ]:
print(analyse_spmf.max_seq(sufficiency_PS2[1], 1, 'No'))

# Goals achievments

## Goals never reached

In [ ]:
print("The following values of variables are never reached in the scenarios:")

never_reached_values = tools.valeursNonAtteintes(state_dict,system)
print(never_reached_values)

In [ ]:
print("so we will never been able to achieve the following goals:")
print(analyse_bdd_general.objnonrealisable(never_reached_values,data_goals))

## Goals statistics

In [ ]:
#Statistics on decisions

statisticsgoals = analyse_bdd_general.statgoals(scenariosgoals)
print(statisticsgoals)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
variable = list(statisticsgoals.keys())
ratio = list(statisticsgoals.values())
width = 0.35

ax.bar(variable,ratio, width, color='powderblue')
ax.bar_label(ax.containers[0], label_type='edge')
plt.ylabel('Ratio de présence avant les objectifs')
plt.xlabel('Decisions')
plt.xticks(rotation=80)
image_format = 'svg' # e.g .png, .svg, etc.
image_name = 'ratio_decision_objectifs.svg'

fig.savefig(image_name, format=image_format, dpi=1200)

plt.show()

## Compromised principles to reach goals

Launch this part if the goals are achieved in at least one scenario

In [ ]:

obj = analyse_bdd_general.ppcompromised_goal(scenariosgoals,system)
print(obj[0])
print(obj[1])

In [ ]:
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])

variables = list(obj[1].keys())

ratio = list(obj[1].values())
width = 0.35

ax.bar(variables,ratio, width, color='indianred')
ax.bar_label(ax.containers[0], label_type='edge')
plt.ylabel('Compromission ratio')
plt.xlabel('Principles')

image_format = 'png' # e.g .png, .svg, etc.
image_name = 'ratio_pp_transgresse_joas.png'

fig.savefig(image_name, format=image_format, dpi=1200)

plt.show()

# Best scenarios

In [ ]:
best = analyse_bdd_general.bestscenario(scenariosgoals, system.pp, system, state_dict, data_goals, data_positions, data_opinions)
print(best)

In [ ]:
#to store the best scenarios according to one principle
principle = "CreationRichesse"
a = tools.allscenariostoreadbetter("bestscenarios_output.txt", best[0].get(principle), state_dict)


In [ ]:
#print the graph representing the number of goals achieved as a function of the decisions taken
scenariosmaxgoals = analyse_bdd_general.scenariosmaxgoalscut(scenarios, state_dict, data_goals, 'Objectif')
scenariosgoals_final = tools.removeduplicate(scenariosmaxgoals)

analyse_bdd_general.graph_objppe(scenariosgoals_final, system, state_dict, data_positions,data_opinions, data_goals, system.pp)